In [ ]:
import pymysql
import pandas as pd

In [ ]:
def get_qry_data_list(qry):
    # MySQL Connection 연결
    conn = pymysql.connect(
        host='localhost', 
        user='etlers', 
        password='wndyd',
        db='stocks', 
        charset='utf8')

    # Connection 으로부터 Cursor 생성
    curs = conn.cursor()
    # SQL문 실행
    curs.execute(qry)
    # 데이타 Fetch
    fetched_list = curs.fetchall()
    # Connection 닫기
    conn.close()
    
    return fetched_list

In [ ]:
def get_jongmok_sum():
    list_cols = [
        "jongmok_cd",
        "jongmok_nm",
        "deal_dt",
        "buy_amount",
        "sell_amount",
        "forgn_amount",
    ]
    # SQL문 실행
    sql = """
    select jongmok_cd 
         , jongmok_nm
         , deal_dt
         , buy_amount
         , sell_amount
         , forgn_amount
      from jongmok_sum
    """
    list_jongmok_sum = get_qry_data_list(sql)    
    df_jongmok_sum = pd.DataFrame(list_jongmok_sum, columns=list_cols)    
    
    return df_jongmok_sum

In [ ]:
def find_target(df_base):
    inc_deal_tf = True
    inc_forgn_tf = True
    pre_day_forgn = 0
    
    df_filtered = df_filtered.sort_values("deal_dt", axis=0, ascending=False).head(5)
    df_filtered = df_filtered.sort_values("deal_dt", axis=0)
    # 기관 거래량이 늘고 외국인 보유가 늘어났다면
    for idx, row in df_filtered.iterrows():
        print(pre_day_forgn, row["forgn_amount"])
        if pre_day_forgn > 0:
            if row["forgn_amount"] <= pre_day_forgn:
                inc_forgn_tf = False
                break
        pre_day_forgn = row["forgn_amount"]

In [ ]:
import pymysql
import pandas as pd

def get_qry_data_list(qry):
    # MySQL Connection 연결
    conn = pymysql.connect(
        host='localhost', 
        user='etlers', 
        password='wndyd',
        db='stocks', 
        charset='utf8')

    # Connection 으로부터 Cursor 생성
    curs = conn.cursor()
    # SQL문 실행
    curs.execute(qry)
    # 데이타 Fetch
    fetched_list = curs.fetchall()
    # Connection 닫기
    conn.close()
    
    return fetched_list

list_cols = [
    "jongmok_cd",
    "jongmok_nm",
    "deal_dt",
    "buy_amount",
    "sell_amount",
    "forgn_amount",
]


def get_jongmok_sum():
    # SQL문 실행
    sql = """
    select jongmok_cd 
         , jongmok_nm
         , deal_dt
         , buy_amount
         , sell_amount
         , forgn_amount
      from jongmok_sum
    """
    list_jongmok_sum = get_qry_data_list(sql)    
    df_jongmok_sum = pd.DataFrame(list_jongmok_sum, columns=list_cols)    
    
    return df_jongmok_sum

def execute():

    def find_target(df_base):
        inc_deal_tf = True
        inc_forgn_tf = True
        read_cnt = 0
        pre_day_forgn = 0
        pre_buy_company = 0
        
        df_filtered = df_base.sort_values("deal_dt", axis=0, ascending=False).head(5)
        df_filtered = df_filtered.sort_values("deal_dt", axis=0)
        # 5일연속 기관 매수량이 늘고 외국인 보유가 늘어났다면
        for idx, row in df_filtered.iterrows():
            if read_cnt > 0:
                if row["forgn_amount"] <= pre_day_forgn:
                    inc_forgn_tf = False
                    break
                if row["buy_amount"] - row["sell_amount"] <= pre_buy_company:
                    inc_deal_tf = False
                    break
            # print(pre_day_forgn, row["forgn_amount"], inc_forgn_tf)
            pre_day_forgn = row["forgn_amount"]
            pre_buy_company = row["buy_amount"] - row["sell_amount"]
            read_cnt += 1

        if inc_deal_tf and inc_forgn_tf:
            result_tf = True
        else:
            result_tf = False

        return result_tf


    df_jongmok_sum = get_jongmok_sum()
    df_jongmok_cd = df_jongmok_sum[["jongmok_cd"]].drop_duplicates()

    jongmok_cnt = 0

    for idx, row in df_jongmok_cd.iterrows():
        df_filtered = df_jongmok_sum.loc[df_jongmok_sum.jongmok_cd == row["jongmok_cd"]]        
        if find_target(df_filtered):
            jongmok_cnt += 1
            if jongmok_cnt == 1:
                df_result = df_filtered
            else:
                df_result = pd.concat([df_result, df_filtered])
            # print(row["jongmok_cd"])

    print(df_result)

if __name__ == "__main__":
    execute()

In [ ]:
execute()